In [ ]:
import numpy as np
import dream
import powder_reduction
import scipp as sc
from scipp import Dim
from scipp.plot import plot

# Testing dream class that generates fake powder dataset
<div class="alert alert-block alert-info">
<b>Information:</b> This notebook shows how to use some of `scipp` functionalities related to the "DREAM" powder diffraction reduction workflow.

<b>Requirements:</b> To run this notebook, you need `matplotlib`, `scipp`, `scipy` installed as well as the Python scripts `powder_reduction.py` and `dream.py` placed in the same folder as this notebook.   
</div>

- Uses the `dream` module to create a fake sparse dataset with a given number of pixels and events.

- The dataset includes a calibration file `dream.cal` which is used to convert the resulting signal to d-spacing.

- At the end the resulting data is plotted using `plotly`.

In [ ]:
# Script for demonstration of event generation
n_pixel = int(2E4)
n_events = int(2E6)

Dream = dream.DreamTest(n_pixel) # Create a fake dream powder diffractometer with detector positions and calibration data
# Display the detector position DataSet and calibration DataSet 
print("Dataset with pixels initialized")
print(Dream.data)
print("Calibration dataset:")
print(Dream.cal)

## Generating a sample
A powder sample with a number of random Bragg peaks are generated. 

Here we create 30 Bragg peaks in the range of 2 to 10 AA with a $\Delta$d over d range between 0.2 and 0.7 %. 

The relative strength of each of these is random as well, varying between 1.0 and 5.0. The sample data will be remembered by the Dream object.

In [ ]:
Dream.generate_random_sample(30, d_range=[2, 10], deltad_over_d_range=[0.2, 0.7], strength_range=[1.0, 5.0])

In [ ]:
print("Generated bragg peak d spacings") # The data can be accessed directly to check
print(Dream.sample_bragg_peaks_d)

## Generating random time of flight data
The fake instrument and sample are now used to generate a requested number of events. 

The wavelength window can be adjusted. 

A simple text based progress bar will be shown unless the keyword argument `verbose` is set to False. 

The resulting data is checked and compared to the ESS timeframe, it would be optimal to have just under 100% coverage.

In [ ]:
print("Generating random sparse data")
Dream.generate_data_pseudo(n_events, wavelength_width=3.5, wavelength_center=3.0)

In [ ]:
print(Dream.data) # Print an overview of the resulting dataset

## Convert time of flight data to dspacing
The raw time of flight data now needs to be converted back to dspacing. This can be done using the calibration data calculated within the Dream object, or without assuming a perfect instrument. 

The two methods will give the same results in this case.

In [ ]:
dspacing = powder_reduction.powder_reduction(Dream.data, calibration=Dream.cal)
print(dspacing) # Show the converted dataset
print(dspacing.dims) # Show the dimensionality of the dspacing dataset

## Plotting the dspacing dataset
Before plotting, the sparse data is converted to histograms for each detector. A few of these are plotted, and it is clear they collect different parts of the dspace range due to their different 2theta. Then the histograms for all detectors are summed and the final dpsacing histogram is plotted.

In [ ]:
N=501
bins = sc.Variable([Dim.DSpacing],
                   values=np.linspace(2.0, 10.0, N).astype(np.float64),
                   unit=sc.units.angstrom)

hist_data = sc.histogram(dspacing, bins)
print(hist_data)

In [ ]:
%matplotlib inline
sc.config.plot.backend = "matplotlib"
plot(hist_data[Dim.Position, 0])
plot(hist_data[Dim.Position, n_pixel-50])

In [ ]:
summed_hist_data = sc.sum(hist_data, Dim.Position)
plot(summed_hist_data)